In [36]:
import pandas as pd

In [37]:
top_delegates_df = pd.read_csv("../Data/Active_Delegates_Data.csv")

In [38]:
top_delegates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   block_time    250 non-null    object 
 1   delegate      250 non-null    object 
 2   voting_power  250 non-null    float64
dtypes: float64(1), object(2)
memory usage: 6.0+ KB


In [39]:
top_delegates_df.head()

,block_time,delegate,voting_power
0,2025-06-17 23:44:09.000 UTC,0x3eee61b92c36e97be6319bf9096a1ac3c04a1466,1.106106e+07
1,2025-06-17 18:48:55.000 UTC,0x2ac5393d1f4be4ef89b45ee2f93d7f20a5cf6d5a,7.026118e+06
2,2025-06-17 20:10:45.000 UTC,0x06ad892ce23c136bbda3a821570343a2af3e2914,5.557348e+06
3,2025-06-17 15:25:15.000 UTC,0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b,5.221303e+06
4,2025-06-17 23:44:01.000 UTC,0x1b686ee8e31c5959d9f5bbd8122a58682788eead,5.040763e+06


In [40]:
votes_df = pd.read_csv("../Data/Agora_Proposal_Votes.csv")

In [41]:
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112560 entries, 0 to 112559
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   proposalId  112560 non-null  object 
 1   address     112560 non-null  object 
 2   timestamp   112560 non-null  object 
 3   support     112560 non-null  object 
 4   weight      112560 non-null  float64
dtypes: float64(1), object(4)
memory usage: 4.3+ MB


In [42]:
votes_df.head()

,proposalId,address,timestamp,support,weight
0,1099608625850195404769236791570123626409652639...,0x60ec4fd8069513f738f3a0f41b9e00c294e74bf3,2025-04-24T18:58:45.000Z,FOR,616478.541200
1,1099608625850195404769236791570123626409652639...,0x010dc5440ad49f9ec0dd325b622d9fd225944ee4,2025-04-24T18:22:19.000Z,FOR,121166.613047
2,1099608625850195404769236791570123626409652639...,0x0331969e189d63fbc31d771bb04ab44227d748d8,2025-04-24T19:02:11.000Z,FOR,106811.043504
3,1099608625850195404769236791570123626409652639...,0x0edfcf9bb8b59b07e53b9b1c4d92351e8d922ab8,2025-04-24T19:26:49.000Z,FOR,736.569032
4,1099608625850195404769236791570123626409652639...,0xbe5afc63dc7b7dcd95631d6e37bbe44e9672a9df,2025-04-24T19:13:37.000Z,FOR,676.288173


In [43]:
# unique proposals
votes_df['proposalId'].unique()

array(['109960862585019540476923679157012362640965263962961594700088102558430487321742',
       '38506287861710446593663598830868940900144818754960277981092485594195671514829',
       '105196850607896626370893604768027381433548036180811365072963268567142002370039',
       '84511922734478887667300419900648701566511387783615524992018614345859900443455',
       '95528263587371532982719325402371584327430753545162858644972401153516332664853',
       '8705916809146420472067303211131851783087744913535435360574720946039078686841',
       '71928632649116715308847337447543955907072794738294227130170691217045092512147',
       '31049359136632781771607732021569520613741907517136820917236339424553298132866',
       '3505139576575581948952533286313165208104296221987341923460133599388956364165',
       '91828175807003897805022589352934414611981179534075907474227649658339164272992'],
      dtype=object)

In [44]:
# Ensure consistent address formatting
top_delegates_df['delegate'] = top_delegates_df['delegate'].str.lower()
votes_df['address'] = votes_df['address'].str.lower()

# Filter votes_df to include only top delegates
filtered_votes = votes_df[votes_df['address'].isin(top_delegates_df['delegate'])]

# Count number of unique proposals each delegate voted on
activity_df = (
    filtered_votes.groupby('address')['proposalId']
    .nunique()
    .reset_index()
    .rename(columns={'address': 'delegate', 'proposalId': 'proposal_participation_count'})
)

# Merge with top_delegates_df to include voting_power, even for inactive ones
final_activity = top_delegates_df.merge(activity_df, on='delegate', how='left')
final_activity['proposal_participation_count'] = final_activity['proposal_participation_count'].fillna(0).astype(int)

# Sort by most active
final_activity = final_activity.sort_values(by='proposal_participation_count', ascending=False)


In [45]:
# Display result
final_activity[['delegate', 'voting_power', 'proposal_participation_count']]

,delegate,voting_power,proposal_participation_count
239,0xcef28ce7884ae973990fb826c1986afaa9319b9a,10211.934669,9
131,0x054becf10c79afa4e60f75b187837be9fb83867d,55115.819890,9
205,0x3f11753c51a8acc79b78b729d477948b4b880da1,14496.836953,9
245,0xc3e8ac233f02d13c1e299e47ae230c205e0177ed,9836.246099,9
233,0x8c57a2288bd9d1a4e02114195678d8eaa215a7a1,10517.215481,9
...,...,...,...
234,0xc540eb6c7b3e44ef8a45db3ad8618404baed88f6,10490.646591,0
244,0x80898b704baa55e7e37f1128fc6ae5836661f54a,9881.418102,0
247,0xd101ee8ab789b5ac467cd0c5343ac596e074e7a9,9651.902232,0
248,0x2a704f87cb047691139c6fec103ffbb10f9c67a2,9563.831504,0


In [46]:
# Count how many delegates did not vote on any proposals
inactive_delegates = final_activity[final_activity['proposal_participation_count'] == 0]

# Total count
num_inactive = len(inactive_delegates)

print(f"Number of top 250 delegates who did NOT vote on any of the proposals: {num_inactive}")

Number of top 250 delegates who did NOT vote on any of the proposals: 114


In [47]:
inactive_delegates[['delegate', 'voting_power']].head()

,delegate,voting_power
101,0x6661db9a250e02444f6f9de670c6d923bcd84cc4,102751.368954
109,0x01e6b50557ef1a2ca58a621998ace3f278e808b8,86109.820782
121,0x8289432acd5eb0214b1c2526a5edb480aa06a9ab,64165.498094
120,0xf51c53b2c184aa43a7d24aa4f0cf13d0e8b56c51,66336.846630
118,0x062a07cbf4848fda67292a96a5e02c97e402233f,67510.415357
